In [156]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_percentage_error

In [157]:
# Load the data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [158]:
#data exploration
print(train.info())
print(train.describe())
print(test.info())
print(test.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230130 entries, 0 to 230129
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   id        230130 non-null  int64  
 1   date      230130 non-null  object 
 2   country   230130 non-null  object 
 3   store     230130 non-null  object 
 4   product   230130 non-null  object 
 5   num_sold  221259 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 10.5+ MB
None
                  id       num_sold
count  230130.000000  221259.000000
mean   115064.500000     752.527382
std     66432.953062     690.165445
min         0.000000       5.000000
25%     57532.250000     219.000000
50%    115064.500000     605.000000
75%    172596.750000    1114.000000
max    230129.000000    5939.000000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98550 entries, 0 to 98549
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ---

In [167]:
#showing null values
print(train.isnull().sum())
print(test.isnull().sum())

id          0
date        0
country     0
store       0
product     0
num_sold    0
dtype: int64
id         0
date       0
country    0
store      0
product    0
dtype: int64


In [160]:
train_copy = train.copy()

In [161]:
#handling missing values
train['num_sold'] = train['num_sold'].dropna()

In [162]:
# Usuń wiersze z brakującymi wartościami w 'num_sold' zarówno z x, jak i y
train = train.dropna(subset=['num_sold'])

# Zaktualizuj zmienne x i y po usunięciu brakujących danych
x = train.drop(columns=['id', 'num_sold'], axis=1)
y = train['num_sold']


In [163]:
#pipeline for preprocessing
numeric_features = x.select_dtypes(include=['int64','float64']).columns
categorical_features = x.select_dtypes(include=['object']).columns
categorical_features = categorical_features[categorical_features != 'date']


print(numeric_features)
print(categorical_features)

Index([], dtype='object')
Index(['country', 'store', 'product'], dtype='object')


In [164]:
# Konwersja kolumny daty na cechy numeryczne
x['year'] = pd.to_datetime(x['date']).dt.year
x['month'] = pd.to_datetime(x['date']).dt.month
x['day'] = pd.to_datetime(x['date']).dt.day

# Usuń oryginalną kolumnę daty
X = x.drop(columns=['date'])


In [165]:
from sklearn.impute import SimpleImputer
#preprocessor 
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Uzupełnianie średnią
    ('scaler', StandardScaler())                 # Skalowanie danych
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Uzupełnianie najczęstszą wartością
    ('onehot', OneHotEncoder(handle_unknown='ignore'))     # OneHotEncoder dla kategorii
])

# Stworzenie ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),      # Numeryczne kolumny
        ('cat', categorical_transformer, categorical_features),  # Kategoryczne kolumny
    ]
)

#models pipeline
models = {
    'Linear Regression': Pipeline(steps=[ #wywołanie Pipeline z dwoma krokami: preprocessor i model
        ('preprocessor', preprocessor),
        ('model', LinearRegression()) #stworzenie instancji modelu
    ]),
    'Random Forest': Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', RandomForestRegressor())
    ]),
    'XGBoost': Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', XGBRegressor())
    ])
}

In [166]:
best_model = None
best_score= float('inf')

for name, model in models.items():
    scores = -cross_val_score(model, x, y, cv=5, scoring='neg_mean_absolute_percentage_error')
    score = np.mean(scores)
    print(f'{name}: {score}')
    if score < best_score:
        best_score = score
        best_model = model

print(f'Best model: {best_model}')
print(f'Best score: {best_score}')

Linear Regression: 4.390755961409231
Random Forest: 0.1528318937132857
XGBoost: 0.15283456747451746
Best model: Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index([], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneH

In [169]:
# Przekształcenie kolumny daty w zbiorze testowym
test['year'] = pd.to_datetime(test['date']).dt.year
test['month'] = pd.to_datetime(test['date']).dt.month
test['day'] = pd.to_datetime(test['date']).dt.day

# Usuń oryginalną kolumnę daty
test_prepared = test.drop(columns=['date'])

#fitting the best model
best_model.fit(x, y)

# Użycie najlepszego modelu do przewidywania
predictions = best_model.predict(test_prepared)

# Jeśli potrzebujesz wyników z kolumną ID:
results = pd.DataFrame({
    'id': test['id'],  # Pobranie ID z testowego DataFrame
    'num_sold': predictions  # Wyniki przewidywań
})

# Zapisanie wyników do pliku CSV (opcjonalne)
results.to_csv('predictions.csv', index=False)

In [170]:
# Konwersja kolumny 'date' na cechy numeryczne
train_copy['year'] = pd.to_datetime(train_copy['date']).dt.year
train_copy['month'] = pd.to_datetime(train_copy['date']).dt.month
train_copy['day'] = pd.to_datetime(train_copy['date']).dt.day
train_copy['day_of_week'] = pd.to_datetime(train_copy['date']).dt.dayofweek

# Podział danych na brakujące i pełne wartości
train_filled = train_copy[train_copy['num_sold'].notna()]
train_missing = train_copy[train_copy['num_sold'].isna()]

# Przygotowanie X i y dla danych bez braków
X_train_filled = train_filled.drop(columns=['num_sold', 'id', 'date'])
y_train_filled = train_filled['num_sold']

# Przygotowanie danych testowych dla brakujących wartości
X_train_missing = train_missing.drop(columns=['num_sold', 'id', 'date'])

# Dopasowanie pipeline i modelu do danych treningowych
best_model.fit(X_train_filled, y_train_filled)

# Predykcja dla brakujących wartości
predicted_values = best_model.predict(X_train_missing)

# Uzupełnienie braków w danych
train_missing['num_sold'] = predicted_values

# Połączenie danych w całość
train_complete = pd.concat([train_filled, train_missing]).sort_index()

# Wyświetlenie wyników
print(train_complete)


            id        date    country                 store  \
0            0  2010-01-01     Canada     Discount Stickers   
1            1  2010-01-01     Canada     Discount Stickers   
2            2  2010-01-01     Canada     Discount Stickers   
3            3  2010-01-01     Canada     Discount Stickers   
4            4  2010-01-01     Canada     Discount Stickers   
...        ...         ...        ...                   ...   
230125  230125  2016-12-31  Singapore  Premium Sticker Mart   
230126  230126  2016-12-31  Singapore  Premium Sticker Mart   
230127  230127  2016-12-31  Singapore  Premium Sticker Mart   
230128  230128  2016-12-31  Singapore  Premium Sticker Mart   
230129  230129  2016-12-31  Singapore  Premium Sticker Mart   

                   product     num_sold  year  month  day  day_of_week  
0        Holographic Goose   104.201479  2010      1    1            4  
1                   Kaggle   973.000000  2010      1    1            4  
2             Kaggle Tie

C:\Users\wiki2\AppData\Local\Temp\ipykernel_23412\2111177391.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_missing['num_sold'] = predicted_values


In [172]:
x = train_complete.drop(columns=['id', 'num_sold'], axis=1)
y = train_complete['num_sold']

numeric_features = x.select_dtypes(include=['int64','float64']).columns
categorical_features = x.select_dtypes(include=['object']).columns
categorical_features = categorical_features[categorical_features != 'date']



# Konwersja kolumny daty na cechy numeryczne
x['year'] = pd.to_datetime(x['date']).dt.year
x['month'] = pd.to_datetime(x['date']).dt.month
x['day'] = pd.to_datetime(x['date']).dt.day

# Usuń oryginalną kolumnę daty
X = x.drop(columns=['date'])



from sklearn.impute import SimpleImputer
#preprocessor 
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Uzupełnianie średnią
    ('scaler', StandardScaler())                 # Skalowanie danych
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Uzupełnianie najczęstszą wartością
    ('onehot', OneHotEncoder(handle_unknown='ignore'))     # OneHotEncoder dla kategorii
])

# Stworzenie ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),      # Numeryczne kolumny
        ('cat', categorical_transformer, categorical_features),  # Kategoryczne kolumny
    ]
)

#models pipeline
models = {
    'Linear Regression': Pipeline(steps=[ #wywołanie Pipeline z dwoma krokami: preprocessor i model
        ('preprocessor', preprocessor),
        ('model', LinearRegression()) #stworzenie instancji modelu
    ]),
    'Random Forest': Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', RandomForestRegressor())
    ]),
    'XGBoost': Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', XGBRegressor())
    ])
}





best_model = None
best_score= float('inf')

for name, model in models.items():
    scores = -cross_val_score(model, x, y, cv=5, scoring='neg_mean_absolute_percentage_error')
    score = np.mean(scores)
    print(f'{name}: {score}')
    if score < best_score:
        best_score = score
        best_model = model

print(f'Best model: {best_model}')
print(f'Best score: {best_score}')

Linear Regression: 5.8826629891620215
Random Forest: 0.1469615661483103
XGBoost: 0.14697052302614383
Best model: Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index([], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   One

In [173]:
# Przekształcenie kolumny daty w zbiorze testowym
test['year'] = pd.to_datetime(test['date']).dt.year
test['month'] = pd.to_datetime(test['date']).dt.month
test['day'] = pd.to_datetime(test['date']).dt.day

# Usuń oryginalną kolumnę daty
test_prepared = test.drop(columns=['date'])

#fitting the best model
best_model.fit(x, y)

# Użycie najlepszego modelu do przewidywania
predictions = best_model.predict(test_prepared)

# Jeśli potrzebujesz wyników z kolumną ID:
results = pd.DataFrame({
    'id': test['id'],  # Pobranie ID z testowego DataFrame
    'num_sold': predictions  # Wyniki przewidywań
})

# Zapisanie wyników do pliku CSV (opcjonalne)
results.to_csv('predictions.csv', index=False)